### Considerações sobre o spacy

Pode ser que ocorra um erro ao executar o treinamento. Se ele ocorrer, executar a linha abaixo para atualizar o spacy.

In [ ]:
#pip install --upgrade spacy

## O código a seguir realiza o treinamento do modelo

In [2]:
from __future__ import unicode_literals, print_function
from spacy.util import minibatch, compounding
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from pathlib import Path
import random
import spacy
import json

TRAIN_DATA = []

mapeamentos = open('mapeamentos_medicamentos.json','r')


fileContent = json.loads(mapeamentos.readline())

for line in fileContent:
  entities = []
  for ents in line["marcacoes"]:
      entities.append((ents["ini"], ents["fim"], ents["classe"]))
      print('entidade', entities)

  item = (line['fa'], {"entities": entities})
  print('item', item)
  TRAIN_DATA.append(item)
  
nlp = spacy.blank("en")  # create blank Language class
print("\n############\nCreated blank 'en' model\n")

# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if "ner" not in nlp.pipe_names:
  ner = nlp.create_pipe("ner")
  nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
  ner = nlp.get_pipe("ner")
  
for _, annotations in TRAIN_DATA:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]

with nlp.disable_pipes(*other_pipes):  # only train NER
  # reset and initialize the weights randomly – but only if we're
  # training a new model
  nlp.begin_training()
  
  for itn in range(100):
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
      texts, annotations = zip(*batch)
      nlp.update(
        texts,  # batch of texts
        annotations,  # batch of annotations
        drop=0.5,  # dropout
        losses=losses
      )
#     print("Losses", losses)

def evaluate(ner_model, examples):
  scorer = Scorer()
  for input_, annot in examples:
    doc_gold_text = ner_model.make_doc(input_)
    gold = GoldParse(doc_gold_text, annot)
    pred_value = ner_model(input_)
    scorer.score(pred_value, gold)
  return scorer.scores

  
output_dir = Path('medicamentos_doc.md')

if not output_dir.exists():
  output_dir.mkdir()

nlp.to_disk(output_dir)
print("Saved model to", output_dir)

ner_model = spacy.load(output_dir)

# results = evaluate(ner_model, TRAIN_DATA)

# results

entidade [(0, 19, 'FAB')]
entidade [(0, 19, 'FAB'), (20, 30, 'MED')]
entidade [(0, 19, 'FAB'), (20, 30, 'MED'), (32, 48, 'CC')]
entidade [(0, 19, 'FAB'), (20, 30, 'MED'), (32, 48, 'CC'), (49, 57, 'TP')]
entidade [(0, 19, 'FAB'), (20, 30, 'MED'), (32, 48, 'CC'), (49, 57, 'TP'), (69, 73, 'QTD')]
item ('Takeda Pharma Ltda. Nesina Met 12,5 MG + 1000 MG COM REV CT BL AL AL X 10', {'entities': [(0, 19, 'FAB'), (20, 30, 'MED'), (32, 48, 'CC'), (49, 57, 'TP'), (69, 73, 'QTD')]})
entidade [(0, 20, 'FAB')]
entidade [(0, 20, 'FAB'), (20, 30, 'MED')]
entidade [(0, 20, 'FAB'), (20, 30, 'MED'), (31, 48, 'CC')]
entidade [(0, 20, 'FAB'), (20, 30, 'MED'), (31, 48, 'CC'), (49, 56, 'TP')]
entidade [(0, 20, 'FAB'), (20, 30, 'MED'), (31, 48, 'CC'), (49, 56, 'TP'), (69, 73, 'QTD')]
item ('Takeda Pharma Ltda. Nesina Met 12,5 MG + 1000 MG COM REV CT BL AL AL X 30', {'entities': [(0, 20, 'FAB'), (20, 30, 'MED'), (31, 48, 'CC'), (49, 56, 'TP'), (69, 73, 'QTD')]})
entidade [(0, 39, 'FAB')]
entidade [(0, 39, 'FAB

item ('Sanofi-Aventis FarmacÃªutica Ltda Menactra SOL INJ CT 5 FA VD TRANS X 0,5ML', {'entities': [(0, 33, 'FAB'), (33, 41, 'MED'), (42, 50, 'CC'), (55, 57, 'TP'), (67, 74, 'QTD')]})
entidade [(0, 36, 'FAB')]
entidade [(0, 36, 'FAB'), (37, 63, 'MED')]
entidade [(0, 36, 'FAB'), (37, 63, 'MED'), (79, 83, 'CC')]
entidade [(0, 36, 'FAB'), (37, 63, 'MED'), (79, 83, 'CC'), (66, 77, 'TP')]
entidade [(0, 36, 'FAB'), (37, 63, 'MED'), (79, 83, 'CC'), (66, 77, 'TP'), (63, 65, 'QTD')]
item ('Cimed IndÃºstria de Medicamentos Ltda Cloridrato de Propranolol 40 Comprimidos40mg', {'entities': [(0, 36, 'FAB'), (37, 63, 'MED'), (79, 83, 'CC'), (66, 77, 'TP'), (63, 65, 'QTD')]})
entidade [(0, 31, 'FAB')]
entidade [(0, 31, 'FAB'), (32, 40, 'MED')]
entidade [(0, 31, 'FAB'), (32, 40, 'MED'), (41, 50, 'CC')]
entidade [(0, 31, 'FAB'), (32, 40, 'MED'), (41, 50, 'CC'), (51, 58, 'TP')]
entidade [(0, 31, 'FAB'), (32, 40, 'MED'), (41, 50, 'CC'), (51, 58, 'TP'), (75, 82, 'QTD')]
item ('Janssen-Cilag FarmacÃªutica Lt

item ('Cosmed Industria de Cosmeticos e Medicamentos S.A. Alivium 600 MG COM REV CT BL AL PLAS TRANS X 4', {'entities': [(0, 50, 'FAB'), (51, 58, 'MED'), (59, 65, 'CC'), (66, 74, 'TP'), (94, 97, 'QTD')]})
entidade [(0, 7, 'FAB')]
entidade [(0, 7, 'FAB'), (8, 18, 'MED')]
entidade [(0, 7, 'FAB'), (8, 18, 'MED'), (19, 23, 'CC')]
entidade [(0, 7, 'FAB'), (8, 18, 'MED'), (19, 23, 'CC'), (24, 27, 'TP')]
entidade [(0, 7, 'FAB'), (8, 18, 'MED'), (19, 23, 'CC'), (24, 27, 'TP'), (46, 50, 'QTD')]
item ('Ems S/A Alprazolam 1 MG COM CT BL AL PLAS INC X 30', {'entities': [(0, 7, 'FAB'), (8, 18, 'MED'), (19, 23, 'CC'), (24, 27, 'TP'), (46, 50, 'QTD')]})
entidade [(0, 6, 'FAB')]
entidade [(0, 6, 'FAB'), (7, 29, 'MED')]
entidade [(0, 6, 'FAB'), (7, 29, 'MED'), (29, 34, 'CC')]
entidade [(0, 6, 'FAB'), (7, 29, 'MED'), (29, 34, 'CC'), (35, 42, 'TP')]
entidade [(0, 6, 'FAB'), (7, 29, 'MED'), (29, 34, 'CC'), (35, 42, 'TP'), (61, 65, 'QTD')]
item ('Germed Montelucaste de Sodio 10 MG COM REV CT BL AL PLAS OPC

entidade [(0, 25, 'FAB'), (26, 37, 'MED'), (38, 69, 'CC'), (70, 73, 'TP')]
entidade [(0, 25, 'FAB'), (26, 37, 'MED'), (38, 69, 'CC'), (70, 73, 'TP'), (92, 96, 'QTD')]
item ('Cellera FarmacÃªutica S.A. Tanderalgin 125 MG + 50 MG + 300 MG + 30 MG COM CT BL AL PLAS INC X 12', {'entities': [(0, 25, 'FAB'), (26, 37, 'MED'), (38, 69, 'CC'), (70, 73, 'TP'), (92, 96, 'QTD')]})
entidade [(0, 25, 'FAB')]
entidade [(0, 25, 'FAB'), (26, 37, 'MED')]
entidade [(0, 25, 'FAB'), (26, 37, 'MED'), (37, 69, 'CC')]
entidade [(0, 25, 'FAB'), (26, 37, 'MED'), (37, 69, 'CC'), (70, 73, 'TP')]
entidade [(0, 25, 'FAB'), (26, 37, 'MED'), (37, 69, 'CC'), (70, 73, 'TP'), (94, 98, 'QTD')]
item ('Cellera FarmacÃªutica S.A. Tanderalgin 125 MG + 50 MG + 300 MG + 30 MG COM CT 2 BL AL PLAS INC X 15', {'entities': [(0, 25, 'FAB'), (26, 37, 'MED'), (37, 69, 'CC'), (70, 73, 'TP'), (94, 98, 'QTD')]})
entidade [(0, 36, 'FAB')]
entidade [(0, 36, 'FAB'), (37, 65, 'MED')]
entidade [(0, 36, 'FAB'), (37, 65, 'MED'), (66, 74, 'CC')

item ('Accord FarmacÃªutica Ltda Bycal 50 MG COM REV CT BL AL PLAS TRANS X 90', {'entities': [(0, 24, 'FAB'), (25, 30, 'MED'), (31, 36, 'CC'), (36, 45, 'TP'), (65, 69, 'QTD')]})
entidade [(0, 24, 'FAB')]
entidade [(0, 24, 'FAB'), (25, 46, 'MED')]
entidade [(0, 24, 'FAB'), (25, 46, 'MED'), (46, 51, 'CC')]
entidade [(0, 24, 'FAB'), (25, 46, 'MED'), (46, 51, 'CC'), (52, 60, 'TP')]
entidade [(0, 24, 'FAB'), (25, 46, 'MED'), (46, 51, 'CC'), (52, 60, 'TP'), (73, 78, 'QTD')]
item ('Accord FarmacÃªutica Ltda Montelucaste de Sodio 4 MG COM MAST CT BL AL AL X 100', {'entities': [(0, 24, 'FAB'), (25, 46, 'MED'), (46, 51, 'CC'), (52, 60, 'TP'), (73, 78, 'QTD')]})
entidade [(0, 24, 'FAB')]
entidade [(0, 24, 'FAB'), (25, 32, 'MED')]
entidade [(0, 24, 'FAB'), (25, 32, 'MED'), (33, 39, 'CC')]
entidade [(0, 24, 'FAB'), (25, 32, 'MED'), (33, 39, 'CC'), (40, 51, 'TP')]
entidade [(0, 24, 'FAB'), (25, 32, 'MED'), (33, 39, 'CC'), (40, 51, 'TP'), (68, 75, 'QTD')]
item ('Accord FarmacÃªutica Ltda Pemtryx 500 

...

item ('Medley Cefalexina 50 MG/ML SUS OR CT FR VD AMB X 100ML + SER DOSAD', {'entities': [(0, 6, 'FAB'), (7, 17, 'MED'), (18, 26, 'CC'), (27, 33, 'TP'), (47, 66, 'QTD')]})
entidade [(0, 6, 'FAB')]
entidade [(0, 6, 'FAB'), (7, 17, 'MED')]
entidade [(0, 6, 'FAB'), (7, 17, 'MED'), (16, 32, 'CC')]
entidade [(0, 6, 'FAB'), (7, 17, 'MED'), (16, 32, 'CC'), (33, 40, 'TP')]
entidade [(0, 6, 'FAB'), (7, 17, 'MED'), (16, 32, 'CC'), (33, 40, 'TP'), (53, 57, 'QTD')]
item ('Sandoz Jalra Met 50 MG + 1000 MG COM REV CT BL AL/AL X 56', {'entities': [(0, 6, 'FAB'), (7, 17, 'MED'), (16, 32, 'CC'), (33, 40, 'TP'), (53, 57, 'QTD')]})
entidade [(0, 6, 'FAB')]
entidade [(0, 6, 'FAB'), (7, 17, 'MED')]
entidade [(0, 6, 'FAB'), (7, 17, 'MED'), (17, 32, 'CC')]
entidade [(0, 6, 'FAB'), (7, 17, 'MED'), (17, 32, 'CC'), (32, 40, 'TP')]
entidade [(0, 6, 'FAB'), (7, 17, 'MED'), (17, 32, 'CC'), (32, 40, 'TP'), (53, 57, 'QTD')]
item ('Sandoz Jalra Met 50 MG + 1000 MG COM REV CT BL AL/AL X 60', {'entities': [(0, 6, 'FAB'

item ('Mabra Peptrat 20 MG CAP DURA LIB RETARD CT BL AL PLAS TRANS  X 30', {'entities': [(0, 5, 'FAB'), (6, 13, 'MED'), (14, 19, 'CC'), (20, 39, 'TP'), (61, 65, 'QTD')]})
entidade [(0, 6, 'FAB')]
entidade [(0, 6, 'FAB'), (7, 14, 'MED')]
entidade [(0, 6, 'FAB'), (7, 14, 'MED'), (15, 20, 'CC')]
entidade [(0, 6, 'FAB'), (7, 14, 'MED'), (15, 20, 'CC'), (21, 24, 'TP')]
entidade [(0, 6, 'FAB'), (7, 14, 'MED'), (15, 20, 'CC'), (21, 24, 'TP'), (43, 47, 'QTD')]
item ('Chiesi Manipal 20 MG COM CT BL PVC/PVDC/AL X 14', {'entities': [(0, 6, 'FAB'), (7, 14, 'MED'), (15, 20, 'CC'), (21, 24, 'TP'), (43, 47, 'QTD')]})
entidade [(0, 6, 'FAB')]
entidade [(0, 6, 'FAB'), (7, 14, 'MED')]
entidade [(0, 6, 'FAB'), (7, 14, 'MED'), (15, 21, 'CC')]
entidade [(0, 6, 'FAB'), (7, 14, 'MED'), (15, 21, 'CC'), (21, 24, 'TP')]
entidade [(0, 6, 'FAB'), (7, 14, 'MED'), (15, 21, 'CC'), (21, 24, 'TP'), (43, 48, 'QTD')]
item ('Chiesi Manipal 20 MG COM CT BL PVC/PVDC/AL X 28Â\xa0', {'entities': [(0, 6, 'FAB'), (7, 14, 'MED'

item ('Eurofarma LaboratÃ³rios S.A. Levofloxacino Hemiidratado 750 MG COM REV CT BL AL PLAS INC X 5', {'entities': [(0, 27, 'FAB'), (28, 54, 'MED'), (55, 61, 'CC'), (62, 69, 'TP'), (88, 91, 'QTD')]})
entidade [(1, 7, 'FAB')]
entidade [(1, 7, 'FAB'), (8, 37, 'MED')]
entidade [(1, 7, 'FAB'), (8, 37, 'MED'), (39, 53, 'CC')]
entidade [(1, 7, 'FAB'), (8, 37, 'MED'), (39, 53, 'CC'), (54, 61, 'TP')]
entidade [(1, 7, 'FAB'), (8, 37, 'MED'), (39, 53, 'CC'), (54, 61, 'TP'), (82, 86, 'QTD')]
item ("'Medley Drospirenona + Etinilestradiol 3 MG + 0,03 MG COM REV CT BL AL PLAS TRANS X 21Â\xa0'", {'entities': [(1, 7, 'FAB'), (8, 37, 'MED'), (39, 53, 'CC'), (54, 61, 'TP'), (82, 86, 'QTD')]})
entidade [(1, 7, 'FAB')]
entidade [(1, 7, 'FAB'), (8, 20, 'MED')]
entidade [(1, 7, 'FAB'), (8, 20, 'MED'), (21, 30, 'CC')]
entidade [(1, 7, 'FAB'), (8, 20, 'MED'), (21, 30, 'CC'), (31, 38, 'TP')]
entidade [(1, 7, 'FAB'), (8, 20, 'MED'), (21, 30, 'CC'), (31, 38, 'TP'), (58, 64, 'QTD')]
item ("'Medley Bimatoprosta 0,

item ('Furp Nitrazepam 5 MG COM CT BL AL PLAS TRANS X 500', {'entities': [(0, 4, 'FAB'), (5, 15, 'MED'), (15, 21, 'CC'), (21, 24, 'TP'), (45, 50, 'QTD')]})
item ('Furp Nitrazepam 5 MG COM CT BL AL PLAS OPC X 10', {'entities': []})
entidade [(0, 38, 'FAB')]
entidade [(0, 38, 'FAB'), (39, 65, 'MED')]
entidade [(0, 38, 'FAB'), (39, 65, 'MED'), (65, 70, 'CC')]
entidade [(0, 38, 'FAB'), (39, 65, 'MED'), (65, 70, 'CC'), (71, 78, 'TP')]
entidade [(0, 38, 'FAB'), (39, 65, 'MED'), (65, 70, 'CC'), (71, 78, 'TP'), (91, 96, 'QTD')]
item ('FundaÃ§Ã£o Para O RemÃ©dio Popular - Furp Bissulfato de Clopidogrel 75 MG COM REV CT BL AL AL X 500', {'entities': [(0, 38, 'FAB'), (39, 65, 'MED'), (65, 70, 'CC'), (71, 78, 'TP'), (91, 96, 'QTD')]})
entidade [(0, 7, 'FAB')]
entidade [(0, 7, 'FAB'), (8, 16, 'MED')]
entidade [(0, 7, 'FAB'), (8, 16, 'MED'), (17, 22, 'CC')]
entidade [(0, 7, 'FAB'), (8, 16, 'MED'), (17, 22, 'CC'), (23, 26, 'TP')]
entidade [(0, 7, 'FAB'), (8, 16, 'MED'), (17, 22, 'CC'), (23, 26, 'TP')

'cp' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


In [3]:
import spacy

from spacy import displacy

nlp = spacy.load('medicamentos_doc.md')

fa_1 = 'Ems Sigma Pharma Ltda Maleato de Enalapril + Hidroclorotiazida 10 MG + 25 MG COM CT BL AL PLAS OPC X 30'

fa_2 = 'Accord Farmacêutica Ltda Bycal 50 MG COM REV CT BL AL PLAS TRANS X 30'

fa_3 = 'Brainfarma Indústria Química e Farmacêutica S.A Quadrilon 0,50 MG/G + 1 MG/G + 10 MG/G + 10 MG/G POM DERM CT BG AL X 15 G'

texto_1 = nlp(fa_1)

texto_2 = nlp(fa_2)

texto_3 = nlp(fa_3)

displacy.render(texto_1, style='ent', jupyter=True)

displacy.render(texto_2, style='ent', jupyter=True)

displacy.render(texto_3, style='ent', jupyter=True)

In [5]:
import pandas as pd
import numpy as np

tags = ['FAB', 'MED', 'CC', 'TP', 'QTD']

apresentacoesDf = pd.read_csv('apresentacoes_medicamentos.csv')

list_apresentacoes = apresentacoesDf.apresentacao.tolist()

list_processadas = []

for apresentacao in list_apresentacoes:
  fa_nlp = nlp(apresentacao)
  
  marks = []
    
  for tag in tags:
    mark_content = [token.text for token in fa_nlp.ents if token.label_ == tag]
    marks.append(mark_content)
    
  list_processadas.append((apresentacao, *marks))


processadasDf = pd.DataFrame(list_processadas, 
                             columns=['apresentacao', 
                                      'fabricante', 
                                      'medicamento', 
                                      'concentracao', 
                                      'tipo', 
                                      'qtd'])

processadasDf.to_csv('apresentacoes_processadas', index=False)

#!cp apresentacoes_processadas.csv

# for entity in texto_1.ents:
#   print(entity.text, ' :: ' , entity.label_)

'cp' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.
